In [1]:
import itertools
import pandas as pd
import numpy as np
import toytree
import toyplot
import arviz as az
import pymc3 as pm

### Tree and dataframe setup

In [2]:
# generate a random tree
NSPECIES = 80
TREE = toytree.rtree.bdtree(
    ntips=NSPECIES,
    seed=666,
).mod.node_scale_root_height(1.0)

# node idxs that delimit several distinct clades on this tree
CLADES = [152, 153, 154, 155]

# draw and color the four major clades
TREE.draw(
    layout='d', 
    width=500,
    tip_labels=False,
    edge_colors=TREE.get_edge_values_mapped({
        j: toytree.colors[i] for i, j in enumerate(CLADES)
    }),
    scalebar=True,
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t2797169c96d54fc1a7e69dc0aa6aec44" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0.0 0.5 1.0

In [5]:
# make group index (gidx)
crown_dict = {i: TREE.get_tip_labels(i) for i in CLADES}
gidx = np.zeros(TREE.ntips, dtype=int)
for tidx, tip in enumerate(TREE.get_tip_labels()):
    for cidx, clade in enumerate(crown_dict):
        if tip in crown_dict[clade]:
            gidx[tidx] = cidx
gidx

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [6]:
# True param values
𝛼_mean = 0.05
𝛼_std = 0.02
𝛽_mean = 3.0
𝛽_std = 0.2
𝜓_mean = 0.0
𝜓_std = 0.33

# 4 different clade effects on rate of RI (used for partial-pooling data)
𝜓_0_mean = 1.0
𝜓_0_std = 0.1
𝜓_1_mean = 0.5
𝜓_1_std = 0.05
𝜓_2_mean = -0.5
𝜓_2_std = 0.05
𝜓_3_mean = -1.0
𝜓_3_std = 0.1

In [7]:
# species dataframe
SPECIES_DATA = pd.DataFrame({
    "gidx": gidx,
    "b": np.random.normal(𝛽_mean, 𝛽_std, TREE.ntips),
    "psi": np.random.normal(𝜓_mean, 𝜓_std, TREE.ntips),
    "psi_x": np.concatenate([
        np.random.normal(𝜓_0_mean, 𝜓_0_std, len(gidx[gidx == 0])),
        np.random.normal(𝜓_1_mean, 𝜓_1_std, len(gidx[gidx == 1])),
        np.random.normal(𝜓_2_mean, 𝜓_2_std, len(gidx[gidx == 2])),
        np.random.normal(𝜓_3_mean, 𝜓_3_std, len(gidx[gidx == 3])),
    ]),
})
SPECIES_DATA.head()

,gidx,b,psi,psi_x
0,0,2.570468,-0.236524,0.997529
1,0,3.292698,-0.624017,0.863576
2,0,2.936227,-0.612795,0.956139
3,0,2.746916,-0.234236,1.115301
4,0,2.767121,0.031270,0.981499


### Generate crossing data

In [29]:
def get_dist(tree, idx0, idx1):
    "returns the genetic distance between two nodes on a tree"
    dist = tree.treenode.get_distance(
        tree.idx_dict[idx0], 
        tree.idx_dict[idx1],
    )
    return dist

# get all combinations of two sampled taxa
a, b = zip(*itertools.combinations(range(NSPECIES), 2))

# organize into DF and get genetic distance between pairs
DATA = pd.DataFrame({
    "sidx0": a,
    "sidx1": b,
    "dist": [(get_dist(TREE, i, j) / 2) for (i, j) in zip(a, b)],
})

DATA['b'] = np.random.normal(𝛽_mean, 𝛽_std, DATA.shape[0])
DATA['velo'] = (
    DATA['b']
    + SPECIES_DATA['psi'][DATA.sidx0].values
    + SPECIES_DATA['psi'][DATA.sidx1].values
)
DATA['velo_x'] = (
    DATA['b']
    + SPECIES_DATA['psi_x'][DATA.sidx0].values
    + SPECIES_DATA['psi_x'][DATA.sidx1].values
)
DATA['intercept'] = np.random.normal(𝛼_mean, 𝛼_std, DATA.shape[0])
# DATA['error'] = np.random.normal(0.0, 𝜎_std, DATA.shape[0])

# get logits
DATA['logit_b'] = (
    1 - (1 - DATA.intercept) * np.exp(-(DATA.b * DATA.dist))
)
DATA['logit'] = (
    1 - (1 - DATA.intercept) * np.exp(-(DATA.velo * DATA.dist))
)
DATA['logit_x'] = (
    1 - (1 - DATA.intercept) * np.exp(-(DATA.velo_x * DATA.dist))
)

# get RI estimates
DATA['RI_pooled'] = np.random.binomial(n=1, p=DATA.logit_b / DATA.logit_b.max())
DATA['RI_unpooled'] = np.random.binomial(n=1, p=DATA.logit / DATA.logit.max())
DATA['RI_partpooled'] = np.random.binomial(n=1, p=DATA.logit_x / DATA.logit_x.max())

DATA.head()

,sidx0,sidx1,dist,b,velo,velo_x,intercept,logit_b,logit,logit_x,RI_pooled,RI_unpooled,RI_partpooled
0,0,1,0.073376,2.929294,2.068754,4.790399,0.011713,0.202858,0.150901,0.304611,0,1,0
1,0,2,0.089748,3.084348,2.235029,5.038016,0.045034,0.275949,0.218601,0.392396,0,0,1
2,0,3,0.129770,3.154197,2.683437,5.267027,0.072539,0.384072,0.345272,0.531776,0,0,1
3,0,4,0.144542,2.732164,2.526910,4.711192,0.060348,0.366920,0.347857,0.524416,0,0,0
4,0,5,0.144542,3.060355,3.305822,5.138901,0.059744,0.395863,0.416922,0.552640,0,1,1


In [30]:
NSAMPLES = 1000
SAMPLE = DATA.sample(NSAMPLES).copy().reset_index(drop=True)
SAMPLE.head()

,sidx0,sidx1,dist,b,velo,velo_x,intercept,logit_b,logit,logit_x,RI_pooled,RI_unpooled,RI_partpooled
0,1,60,1.000000,3.303329,2.290247,3.732741,0.070190,0.965820,0.905865,0.977752,1,0,1
1,26,67,1.000000,2.785263,3.120073,3.011441,0.059715,0.941972,0.958483,0.953718,1,1,1
2,8,27,0.438896,3.190428,3.033748,5.403544,0.025868,0.759846,0.742751,0.909083,0,1,1
3,24,61,1.000000,3.263680,3.032420,4.014547,0.017115,0.962407,0.952626,0.982258,1,1,1
4,38,63,1.000000,2.915690,3.156976,2.882806,0.039309,0.947963,0.959119,0.946223,1,1,1


### Visualize data

In [10]:
def logit_plot(dist, logit, RI):
    canvas = toyplot.Canvas(width=500, height=250)
    ax0 = canvas.cartesian(
        label="pooled data (function)",
        xlabel="Genetic dist.",
        ylabel="Logit function",
        grid=(1, 2, 0),
    )
    ax1 = canvas.cartesian(
        label="pooled data (observation)",
        xlabel="Genetic dist.",
        ylabel="RI",
        grid=(1, 2, 1),
    )

    # points are jittered on x-axis for visibility
    ax0.scatterplot(
        dist,
        logit,
        size=5,
        opacity=0.33,
        color=toyplot.color.Palette()[0],
    );
    ax1.scatterplot(
        dist,
        RI,
        size=10,
        opacity=0.2,
        marker="|",
        mstyle={
            "stroke": toyplot.color.Palette()[1],
            "stroke-width": 3,
        },
    );
    return canvas, (ax0, ax1)

In [31]:
logit_plot(SAMPLE.dist, SAMPLE.logit_b, SAMPLE.RI_pooled);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t9630222449f249c1bbdee388badfb90f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:rgb(40%,76.1%,6

In [32]:
logit_plot(SAMPLE.dist, SAMPLE.logit, SAMPLE.RI_unpooled);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="tb522c2b7b2d641ddab15b37abb0f1a49" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:rgb(40%,76.1%,64.7%);s

In [33]:
logit_plot(SAMPLE.dist, SAMPLE.logit_x, SAMPLE.RI_partpooled);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t1c8cfd4a94304973a4085e5d5dcef381" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> </

### Define models

In [ ]:
1 - (1 - DATA.intercept) * np.exp(-(DATA.velo * DATA.dist))

In [34]:
def pooled_logistic(x, y, **kwargs):
    
    # define model
    with pm.Model() as model:  

        # parameters and error
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        𝛽 = pm.Normal('𝛽', mu=0., sigma=10., shape=1)
        
        # link function
        effect = 1 - (1 -𝛼) * np.exp(-(𝛽 * x))
        logit = pm.Deterministic("logit", pm.invlogit(effect))
        
        # data likelihood
        y = pm.Bernoulli("y", p=logit, observed=y)
        
        # sample posterior, skip burnin
        trace = pm.sample(**kwargs)[1000:]
    
        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

In [36]:
def unpooled_logistic(x, y, idx0, idx1, **kwargs):
    
    # define model
    with pm.Model() as model:
        
        # indexers
        sidx0 = pm.Data("spp_idx0", idx0.values)
        sidx1 = pm.Data("spp_idx1", idx1.values)

        # parameters and error
        𝜓_mean = pm.Normal('𝜓_mean', mu=0., sigma=5., shape=1)
        𝜓_std = pm.HalfNormal('𝜓_std', 5., shape=1)
        𝜓_offset = pm.Normal('𝜓_offset', mu=0, sigma=1., shape=TREE.ntips)
        𝜓 = pm.Deterministic('𝜓', 𝜓_mean + 𝜓_std * 𝜓_offset)
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        𝛽 = pm.Normal('𝛽', mu=0., sigma=10., shape=1)
        
        # link function
        effect = 1 - (1 -𝛼) * np.exp(-((𝛽 + 𝜓[sidx0] + 𝜓[sidx1]) * x))
        logit = pm.Deterministic("logit", pm.invlogit(effect))
        
        # data likelihood
        y = pm.Bernoulli("y", p=logit, observed=y)
        
        # sample posterior, skip burnin
        trace = pm.sample(**kwargs)[1000:]
    
        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

In [37]:
def partpooled_logistic(x, y, idx0, idx1, gidx, **kwargs):
    
    # define model
    with pm.Model() as model:
        
        # indexers
        sidx0 = pm.Data("spp_idx0", idx0)
        sidx1 = pm.Data("spp_idx1", idx1)
        gidx = pm.Data("gidx", gidx)

        # parameters and error
        𝜓_mean = pm.Normal('𝜓_mean', mu=0., sigma=5., shape=4)
        𝜓_std = pm.HalfNormal('𝜓_std', 5., shape=4)
        𝜓_offset = pm.Normal('𝜓_offset', mu=0, sigma=1., shape=TREE.ntips)
        𝜓 = pm.Deterministic('𝜓', 𝜓_mean[gidx] + 𝜓_std[gidx] * 𝜓_offset)
        𝛽 = pm.Normal('𝛽', mu=0., sigma=10., shape=1)
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        
        # linear model prediction
        effect = 1 - (1 -𝛼) * np.exp(-((𝛽 + 𝜓[sidx0] + 𝜓[sidx1]) * x))
        logit = pm.Deterministic("logit", pm.invlogit(effect))
        
        # data likelihood (normal distributed errors)
        y = pm.Bernoulli("y", p=logit, observed=y)

        # sample posterior, skip burnin
        trace = pm.sample(**kwargs)[1000:]

        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

### Functions to plot results

In [17]:
def toytrace(trace, var_names, titles):
    """
    Plot posterior trace with toyplot
    """
    nvars = len(var_names)
    
    # setup canvase
    canvas = toyplot.Canvas(width=500, height=200 * nvars)
    
    # store axes
    axes = []
    
    # iter over params
    for pidx, param in enumerate(var_names):
        
        # get param posterior
        posterior = trace.get_values(param)
        
        # setup axes 
        ax = canvas.cartesian(grid=(nvars, 1, pidx))
        ax.y.show = False
        ax.x.spine.style = {"stroke-width": 1.5}
        ax.x.ticks.labels.style = {"font-size": "12px"}
        ax.x.ticks.show = True
        ax.x.label.text = f"param='{titles[pidx]}'"        
        
        # iterate over shape of param
        for idx in range(posterior.shape[1]):
            mags, bins = np.histogram(posterior[:, idx], bins=100)
            ax.plot(bins[1:], mags, stroke_width=2, opacity=0.6)
        axes.append(ax)
    return canvas, axes

In [18]:
import scipy.stats as stats

def draw_velocity_dists(trace, baseline=0.15):
    """
    Draw the clade velocities as gaussians
    """
    canvas = toyplot.Canvas(width=350, height=300)
    axes = canvas.cartesian(xlabel="Relative velocity of reproductive isolation")
    marks = []
    base = 0
    for i in range(trace['𝜓_mean'].shape[1]):
        
        loc = trace['𝜓_mean'][:, i].mean()
        scale = trace['𝜓_std'][:, i].mean()
        interval = stats.norm.interval(0.995, loc, scale)
        points = np.linspace(interval[0], interval[1], 100)
        mark = axes.fill(
            points, 
            stats.norm.pdf(points, loc=loc, scale=scale), 
            style={
                "fill-opacity": 0.45,
                "stroke": 'black',
                "stroke-opacity": 1.0,
                "stroke-width": 1,
            },
            baseline=np.repeat(base, 100),
        )
        marks.append(mark)
        axes.hlines(base, style={"stroke-dasharray": "5,5", 'stroke-width': 1})
        base += baseline
        
    axes.y.show = False
    axes.x.ticks.locator = toyplot.locator.Extended(only_inside=True)
    axes.x.ticks.show = True
    return canvas, axes, marks

In [ ]:
# MCMC sampler kwargs
sample_kwargs = dict(
    tune=10000,
    draws=10000,
    target_accept=0.99,
    return_inferencedata=False,
    progressbar=True,
)

### Run three datasets under pooled model

In [38]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_pooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# pooled model
pooled_model_pooled_data = pooled_logistic(*model_args[:2], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛽, 𝛼]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 566 seconds.
There were 9 divergences after tuning. Increase `target_accept` or reparameterize.
There were 12 divergences after tuning. Increase `target_accept` or reparameterize.
There were 12 divergences after tuning. Increase `target_accept` or reparameterize.
There were 28 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


In [ ]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_unpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# pooled model
pooled_model_unpooled_data = pooled_logistic(*model_args[:2], **sample_kwargs)

In [ ]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_partpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# pooled model
pooled_model_partpooled_data = pooled_logistic(*model_args[:2], **sample_kwargs)

### Run three datasets under unpooled model

In [ ]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_pooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
unpooled_model_pooled_data = unpooled_logistic(*model_args[:4], **sample_kwargs)

In [ ]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_unpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
unpooled_model_unpooled_data  = unpooled_logistic(*model_args[:4], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛽, 𝛼, 𝜓_offset, 𝜓_std, 𝜓_mean]


In [70]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_partpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
unpooled_model_partpooled_data = unpooled_logistic(*model_args[:4], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛽, 𝛼, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 22826 seconds.
There were 4 divergences after tuning. Increase `target_accept` or reparameterize.
There were 7 divergences after tuning. Increase `target_accept` or reparameterize.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
There were 4 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


### Run three datasets under partpooled model

In [71]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_pooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
partpooled_model_pooled_data = partpooled_logistic(*model_args, **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛼, 𝛽, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 9135 seconds.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
There were 5 divergences after tuning. Increase `target_accept` or reparameterize.
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


In [ ]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_unpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
partpooled_model_unpooled_data = partpooled_logistic(*model_args, **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛼, 𝛽, 𝜓_offset, 𝜓_std, 𝜓_mean]


In [28]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_partpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
partpooled_model_partpooled_data = partpooled_logistic(*model_args, **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛼, 𝛽, 𝜓_offset, 𝜓_std, 𝜓_mean]


KeyError: 0

In [144]:
partpooled_sub['stats']

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
𝜓_mean[0],1.558,2.253,-2.615,5.883,0.020,0.014,13045.0,13045.0,13047.0,19597.0,1.0
𝜓_mean[1],1.380,2.337,-3.002,5.790,0.020,0.014,13576.0,13576.0,13576.0,20055.0,1.0
𝜓_mean[2],-0.070,2.250,-4.278,4.226,0.020,0.014,12972.0,12972.0,12977.0,19782.0,1.0
𝜓_mean[3],-1.312,2.252,-5.568,2.946,0.020,0.014,13078.0,13078.0,13083.0,19707.0,1.0
𝜓_offset[0],0.348,0.947,-1.393,2.169,0.004,0.005,66932.0,15121.0,66978.0,23293.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
logit[995],0.988,0.011,0.971,1.000,0.000,0.000,34600.0,34600.0,34485.0,28201.0,1.0
logit[996],0.986,0.014,0.964,1.000,0.000,0.000,39528.0,39528.0,47966.0,30431.0,1.0
logit[997],0.728,0.058,0.617,0.833,0.000,0.000,44869.0,44869.0,45744.0,32264.0,1.0
logit[998],0.928,0.032,0.870,0.981,0.000,0.000,48364.0,48364.0,56145.0,29106.0,1.0


In [145]:
toytrace(partpooled_sub['trace'], ['𝜓_mean', '𝜓_offset', '𝜓'], ['psi-mean', 'psi-offset', 'psi-spp']);

<svg class="toyplot-canvas-Canvas" height="600.0px" id="t7f9f5f9ce60345f48ec7c1da55992653" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 600.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 110.09128780189101 149.9198717948718 L 113.39308783398756 150.0 L 116.69488786608413 149.9198717948718 L 119.9966878981807 149.8397435897436 L 123.29848793027726 149.8397435897436 L 126.60028796237381 149.9198717948718 L 129.90208799447038 149.9198717948718 L 133.20388802656694 149.67948717948718 L 136.5056880586635 149.59935897435898 L 139.80748809076005 149.9198717948718 L 143.1092881228566 149.51923076923077 L 146.41108815495318 149.7596153846154 L 149.71288818704977 149.43910256410257 L 153.0146882191463 148.47756410256412 L 156.31648825124287 147.83653846153845 L 159.61828828333944 147.91666666666666 L 162.920088315436 147.59615384615384 L 166.22188834753257 147.83653846153845 L 169.5236883796291 146.7948717948718 L 172.8254884117257 145.83333333333334 L 176.12728844382227 145.03205128205127 L 179.42908847591883 143.75 L 182.73088850801537 143.34935897435898 L 186.03268854011196 141.50641025641025 L 189.33448857220847 138.94230769230768 L 192.63628860430507 136.37820512820514 L 195.93808863640163 137.2596153846154 L 199.2398886684982 135.57692307692307 L 202.54168870059476 131.25 L 205.84348873269133 127.80448717948718 L 209.1452887647879 126.3621794871795 L 212.44708879688443 125.0801282051282 L 215.748888828981 120.03205128205127 L 219.05068886107753 119.47115384615384 L 222.35248889317415 110.17628205128204 L 225.6542889252707 108.65384615384616 L 228.95608895736729 105.68910256410257 L 232.25788898946385 101.7628205128205 L 235.5596890215604 96.07371794871796 L 238.86148905365695 90.30448717948718 L 242.16328908575346 83.97435897435898 L 245.46508911785006 80.20833333333334 L 248.76688914994662 79.00641025641025 L 252.06868918204316 72.11538461538461 L 255.37048921413978 71.23397435897436 L 258.6722892462363 66.42628205128204 L 261.9740892783329 71.7948717948718 L 265.2758893104294 62.33974358974359 L 268.57768934252596 58.41346153846154 L 271.87948937462255 56.00961538461539 L 275.18128940671915 62.33974358974359 L 278.48308943881574 58.092948717948715 L 281.7848894709122 62.82051282051282 L 285.0866895030088 59.61538461538461 L 288.3884895351054 61.93910256410257 L 291.6902895672019 70.03205128205128 L 294.9920895992985 63.621794871794876 L 298.2938896313951 69.07051282051282 L 301.59568966349167 78.44551282051282 L 304.8974896955882 74.83974358974359 L 308.1992897276848 77.40384615384616 L 311.50108975978134 82.37179487179486 L 314.8028897918779 84.93589743589743 L 318.10468982397447 94.07051282051282 L 321.406489856071 94.95192307692308 L 324.7082898881676 102.24358974358972 L 328.01008992026414 105.76923076923077 L 331.31188995236073 111.05769230769232 L 334.61368998445727 114.50320512820514 L 337.9154900165538 119.15064102564102 L 341.21729004865034 121.63461538461539 L 344.51909008074693 123.15705128205127 L 347.8208901128435 126.7628205128205 L 351.1226901449401 132.05128205128204 L 354.4244901770366 134.4551282051282 L 357.7262902091332 135.9775641025641 L 361.0280902412298 137.41987179487177 L 364.32989027332627 139.1025641025641 L 367.63169030542286 142.2275641025641 L 370.93349033751946 142.06730769230768 L 374.23529036961605 145.99358974358975 L 377.53709040171253 144.31089743589743 L 380.8388904338092 143.9102564102564 L 384.1406904659057 146.6346153846154 L 387.4424904980022 147.0352564102564 L 390.74429053009885 147.75641025641025 L 394.0460905621954 148.47756410256412 L 397.347890594292 148.47756410256412 L 400.64969062638846 148.31730769230768 L 403.951

In [146]:
# show plot of TRUE vs. ESTIMATED rates
c, a, m = toyplot.scatterplot(
    partpooled_sub['trace']['𝜓'].mean(axis=0),         # estimated
    SPECIES_DATA['psi_x'],                             # true
    width=400,
    height=250,
    xlabel="ESTIMATED species velocity",
    ylabel="TRUE species velocity",
    color=[toyplot.color.Palette()[i] for i in SPECIES_DATA.gidx],
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t9df28661fa6048888438619ce2d3232a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 250.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> -2 -1 0 1 2 ESTIMATED species velocity -1 0 1 TRUE species velocity

In [190]:
draw_velocity_dists(partpooled_sub['trace'], baseline = 1.2);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t584c07f44c734869828950326551561a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <polygon points="125.5744499492483,250.0 126.82841083834217,250.0 128.08237172743605,250.0 129.33633261652992,250.0 130.59029350562378,250.0 131.84425439471764,250.0 133.09821528381153,250.0 134.35217617290536,250.0 135.60613706199922,250.0 136.8600979510931,250.0 138.114058840187,250.0 139.36801972928086,250.0 140.62198061837472,250.0 141.87594150746858,250.0 143.12990239656244,250.0 144.38386328565633,250.0 145.6378241747502,250.0 146.89178506384405,250.0 148.14574595293792,250.0 149.39970684203178,250.0 150.65366773112567,250.0 151.90762862021953,250.0 153.1615895093134,250.0 154.41555039840728,250.0 155.6695112875011,250.0 156.92347217659497,250.0 158.17743306568886,250.0 159.43139395478272,250.0 160.6853548438766,250.0 161.93931573297047,250.0 163.19327662206433,250.0 164.4472375111582,250.0 165.70119840025205,250.0 166.95515928934594,250.0 168.2091201784398,250.0 169.4630810675337,250.0 170.71704195662755,250.0 171.9710028457214,250.0 173.22496373481528,250.0 174.47892462390914,250.0 175.732885513003,250.0 176.98684640209686,250.0 178.24080729119075,250.0 179.4947681802846,250.0 180.7487290693785,250.0 182.00268995847233,250.0 183.2566508475662,250.0 184.51061173666008,250.0 185.76457262575394,250.0 187.01853351484783,250.0 188.2724944039417,250.0 189.52645529303553,250.0 190.78041618212941,250.0 192.03437707122328,250.0 193.28833796031716,250.0 194.54229884941103,250.0 195.7962597385049,250.0 197.05022062759875,250.0 198.3041815166926,250.0 199.5581424057865,250.0 200.81210329488036,250.0 202.06606418397422,250.0 203.32002507306808,250.0 204.57398596216197,250.0 205.82794685125583,250.0 207.0819077403497,250.0 208.33586862944358,250.0 209.58982951853747,250.0 210.8437904076313,250.0 212.09775129672516,250.0 213.35171218581903,250.0 214.6056730749129,250.0 215.85963396400678,250.0 217.1135948531006,250.0 218.36755574219447,250.0 219.62151663128836,250.0 220.87547752038225,250.0 222.1294384094761,250.0 223.38339929856997,250.0 224.63736018766383,250.0 225.89132107675775,250.0 227.14528196585158,250.0 228.39924285494547,250.0 229.6532037440393,250.0 230.90716463313316,250.0 232.16112552222702,250.0 233.41508641132089,250.0 234.66904730041475,250.0 235.92300818950864,250.0 237.1769690786025,250.0 238.4309299676964,250.0 239.68489085679025,250.0 240.9388517458841,250.0 242.192812634978,250.0 243.44677352407186,250.0 244.70073441316575,250.0 245.9546953022596,250.0 247.20865619135344,250.0 248.4626170804473,250.0 249.71657796954116,250.0 249.71657796954116,249.4750801830467 248.4626170804473,249.3854942943443 247.20865619135344,249.28292880062446 245.9546953022596,249.16593086745948 244.70073441316575,249.032958242902 243.44677352407186,248.8823860473273 242.192812634978,248.71251608551677 240.9388517458841,248.52158888945337 239.68489085679025,248.3077986671614 238.4309299676964,248.0693112898063 237.1769690786025,247.8042853959786 235.92300818950864,247.51089662874148 234.66904730041475,247.18736494811205 233.41508641132089,246.83198488005388 232.16112552222702,246.44315847408527 230.90716463313316,246.0194306469942 229.6532037440393,245.5595264920478 228.39924285494547,245.06239003405332 227.14528196585158,244.52722381357555 225.89132107675775,243.9535285917426 224.63736018766383,243.34114238377992 223.38339929856997,242.6902779582103 222.1294384094761,242.00155788303047 220.87547752038225,241.27604616346812 219.62151663128836,240.51527550118925 218.36755574219447,239.72126921469416 217

### Assess model fit

In [165]:
def rmse(predictions, targets):
    differences = predictions - targets                       #the DIFFERENCEs.
    differences_squared = differences ** 2                    #the SQUAREs of ^
    mean_of_differences_squared = differences_squared.mean()  #the MEAN of ^
    rmse_val = np.sqrt(mean_of_differences_squared)           #ROOT of ^
    return rmse_val   

In [166]:
def aicm(mean, variance):
    return 2*mean - 2*variance

In [167]:
rmse(unpooled_sub['trace']['𝜓'].mean(axis=0), SPECIES_DATA['psi'])

0.9384409617057597

In [168]:
rmse(partpooled_sub['trace']['𝜓'].mean(axis=0), SPECIES_DATA['psi_x'])

0.6105422350829149

In [177]:
aicm(unpooled_sub['trace']['𝜓'].mean(), unpooled_sub['trace']['𝜓'].var())

-24.454868204001738

In [179]:
# Raftery et al. (2007) holds that this value is better.  R+M disagree?
aicm(partpooled_sub['trace']['𝜓'].mean(), partpooled_sub['trace']['𝜓'].var())

-12.49733802346165

In [218]:
pooled_az = az.from_pymc3(trace = pooled_sub['trace'], model = pooled_sub['model'])
unpooled_az = az.from_pymc3(trace = unpooled_sub['trace'], model = unpooled_sub['model'])
partpooled_az = az.from_pymc3(trace = partpooled_sub['trace'], model = partpooled_sub['model'])

In [227]:
az.loo(pooled_az, pointwise = True)

Computed from 36000 by 1000 log-likelihood matrix

         Estimate       SE
elpd_loo  -256.22    18.93
p_loo        2.00        -
------

Pareto k diagnostic values:
                         Count   Pct.
(-Inf, 0.5]   (good)     1000  100.0%
 (0.5, 0.7]   (ok)          0    0.0%
   (0.7, 1]   (bad)         0    0.0%
   (1, Inf)   (very bad)    0    0.0%

In [228]:
az.loo(unpooled_az, pointwise = True)

Computed from 36000 by 1000 log-likelihood matrix

         Estimate       SE
elpd_loo  -272.70    20.02
p_loo       21.57        -
------

Pareto k diagnostic values:
                         Count   Pct.
(-Inf, 0.5]   (good)     1000  100.0%
 (0.5, 0.7]   (ok)          0    0.0%
   (0.7, 1]   (bad)         0    0.0%
   (1, Inf)   (very bad)    0    0.0%

In [229]:
az.loo(partpooled_az, pointwise = True)

/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:684: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  "Estimated shape parameter of Pareto distribution is greater than 0.7 for "


Computed from 36000 by 1000 log-likelihood matrix

         Estimate       SE
elpd_loo  -250.38    19.28
p_loo       19.09        -

There has been a warning during the calculation. Please check the results.
------

Pareto k diagnostic values:
                         Count   Pct.
(-Inf, 0.5]   (good)      995   99.5%
 (0.5, 0.7]   (ok)          4    0.4%
   (0.7, 1]   (bad)         1    0.1%
   (1, Inf)   (very bad)    0    0.0%

In [222]:
az.compare({"pooled": pooled_az, 
            "unpooled": unpooled_az, 
            "partpooled": partpooled_az})

/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:684: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  "Estimated shape parameter of Pareto distribution is greater than 0.7 for "


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
partpooled,0,-250.382,19.0949,0,0.523341,19.7962,0,True,log
pooled,1,-256.22,1.99842,5.83739,0.366993,19.7882,25.4467,False,log
unpooled,2,-272.704,21.5726,22.322,0.109666,18.6714,26.2816,False,log
